In [ ]:
import tensorflow as tf
with tf.device("/job:worker/task:1"):
        x = tf.Variable(tf.ones([2, 2]))
        y = tf.Variable(tf.ones([2, 2]))

with tf.device("/job:worker/task:0"):
    z = tf.matmul(x, y) + x

with tf.device("/job:worker/task:1"):
    z = tf.matmul(z, x) + x

with tf.Session("grpc://localhost:2223") as sess:
    sess.run(tf.global_variables_initializer())
    val = sess.run(z)
    print(val)


In [1]:
import os
os.getpid()

18742

In [ ]:
import tensorflow as tf
with tf.Session("grpc://localhost:2223") as sess:
    sess.run(tf.global_variables_initializer())
    val = sess.run(z)
    print(val)

In [ ]:
import tensorflow as tf
import numpy as np 

np.random.seed(5)  
sx = np.linspace(-1, 1, 100)  
sy = 2 * sx + 1.0 + np.random.randn(100) * 0.4
k = tf.Variable(1.0, name='k')
b = tf.Variable(0, name='b',dtype=tf.float32)

x = tf.placeholder(dtype=tf.float32,name='x')
y = tf.placeholder(dtype=tf.float32,name='y')

yp = tf.add(tf.multiply(x, k), b, name='yp')
# 训练模型，设置训练参数(迭代次数、学习率)
train_epoch = 10
rate = 0.05
# 定义均方差为损失函数
loss = tf.reduce_mean(tf.square(y - yp))
# 定义梯度下降优化器，并传入参数学习率和损失函数
optimizer = tf.train.GradientDescentOptimizer(rate).minimize(loss)
init = tf.global_variables_initializer()

with tf.Session("grpc://localhost:2222") as sess:
    with tf.device("/job:worker/task:0"):
        sess.run(init)
        for _ in range(10):
            for x1, y1 in zip(sx, sy):
                sess.run(optimizer, feed_dict={x: x1, y: y1})
            
        print("k : %s" % k.eval(session=sess))
        print("b : %s" % b.eval(session=sess))

关闭脚命令
ps -ef | grep python| grep 脚本名称 | awk {'print $2'} | xargs kill